As of 8 March, the best model according to recall@30 (validation set) is registered in wandb in the project "thesis_basic_deterministic_model_tunning" and it is called "pleasant-music-50"

The config is as follow:
- type: lstm
- hidden_size = 100
- lr: 0.01
- layers = 1
with:
- batch_size = 64
- dropout: 0.1
- epochs: 15
and:
- recall@30 (validation): 0.7208
- recall@30 (test): 
- precision@30 (validation): 
- precision@30 (test): 


In this notebook i will create a pickle of this model to be used for later experiments

# Change working directory to parent

Assumes this script is currently placed in a child folder of the main project.

In [106]:
import os
cwd = os.getcwd()

# protection against running this cell multiple times
assert os.path.dirname(cwd).split('/')[-1] == 'master-thesis','Oops, directory already changed previously as indended. Ignoring...'

# change working directory (if assert passed)
new_cwd = os.path.dirname(cwd) # parent directory
os.chdir(new_cwd)

AssertionError: Oops, directory already changed previously as indended. Ignoring...

In [2]:
# show all outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
model_name = 'pleasant-music-50'

In [4]:
import os
import json

from rnn_utils import DiagnosesDataset, split_dataset, MYCOLLATE
from rnn_utils import RNN, train_one_epoch, eval_model

from config import Settings; settings = Settings()


import torch
from torch.utils.data import Dataset, DataLoader, random_split

from sklearn.model_selection import ParameterGrid, ParameterSampler

import numpy as np
import pandas as pd

import wandb

# Parameters

In [5]:
dataset_id = 'diag_only'

# Reproducibility

In [6]:
# Reproducibility
seed = settings.random_seed
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Create dataset

In [7]:
# hyperparameters of best model
grouping = 'ccs'
batch_size=64

In [8]:
dataset_folder = os.path.join(settings.data_base,settings.model_ready_dataset_folder,dataset_id)
print('dataset at',dataset_folder)

dataset at data/model_ready_dataset/diag_only


In [9]:
dataset = DiagnosesDataset(os.path.join(dataset_folder,'dataset.json'),grouping)

train_dataset = DiagnosesDataset(os.path.join(dataset_folder,'dataset/train_subset.json'),grouping)
val_dataset = DiagnosesDataset(os.path.join(dataset_folder,'dataset/val_subset.json'),grouping)
test_dataset = DiagnosesDataset(os.path.join(dataset_folder,'dataset/test_subset.json'),grouping)


len(train_dataset)
len(val_dataset)
len(test_dataset)


train_dataloader = DataLoader(train_dataset,batch_size=batch_size,collate_fn=MYCOLLATE(dataset),shuffle=True)
val_dataloader = DataLoader(val_dataset,batch_size=batch_size,collate_fn=MYCOLLATE(dataset)) #batch_size here is arbitrary and doesn't affect total validation speed
test_dataloader = DataLoader(test_dataset,batch_size=batch_size,collate_fn=MYCOLLATE(dataset))

5249

1125

1125

# Define model

## Hyperparameters

In [10]:
# remaining hyperparameters of best model
input_size = next(iter(train_dataloader))['target_sequences']['sequence'].shape[2]
hidden_size = 75
num_layers = 1
n_labels = input_size
rnn_type = 'lstm'

lr = 0.01

n_labels = input_size
epochs = 10

# Train

In [11]:
model = RNN(input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            n_labels=n_labels,
            model=rnn_type,
            )
    
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

criterion = torch.nn.BCEWithLogitsLoss(reduction='none')

for epoch in range(1,epochs+1):
    loss = train_one_epoch(model, train_dataloader, epoch, criterion, optimizer);
    print(f"epoch {epoch}\t| loss {loss}\t")

epoch 1	| loss 0.18619780466858163	
epoch 2	| loss 0.12913943243673048	
epoch 3	| loss 0.12414237461894391	
epoch 4	| loss 0.1200773758162935	
epoch 5	| loss 0.11707099177033069	
epoch 6	| loss 0.11408706395381904	
epoch 7	| loss 0.11192040504461312	
epoch 8	| loss 0.1100557976099382	
epoch 9	| loss 0.10944202273006899	
epoch 10	| loss 0.10679219591330333	


# Eval

confirm it has good results

In [12]:
from rnn_utils import get_prediction_thresholds, outs2df,compute_metrics

In [13]:
train_outs,train_golden = outs2df(model,train_dataloader,dataset,return_golden=True)
val_outs,val_golden = outs2df(model,val_dataloader,dataset,return_golden=True)
test_outs,test_golden = outs2df(model,test_dataloader,dataset,return_golden=True)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [107]:
train_outs.head(1)

,,diag_0,diag_1,diag_2,diag_3,diag_4,diag_5,diag_6,diag_7,diag_8,diag_9,...,diag_262,diag_263,diag_264,diag_265,diag_266,diag_267,diag_268,diag_269,diag_270,diag_271
pat_id,adm_index,,,,,,,,,,,,,,,,,,,,,
17,1,0.001214,0.108292,0.076243,0.029526,0.000564,0.013792,0.013004,0.010701,0.00111,0.025458,...,0.000399,0.002593,0.001516,0.002587,0.183489,0.100942,0.003286,0.002262,0.016034,0.030779


In [153]:
pd.Series([1,2,3],index=['a_art','b_art','c']).filter(like="_art")

a_art    1
b_art    2
dtype: int64

In [154]:
# check admissions are in order to later on produce correctly the last admission
assert train_outs.reset_index().groupby('pat_id').adm_index.apply(lambda x: x.is_monotonic_increasing).all()

operations = {'last admission': lambda df: df.reset_index().groupby('pat_id').apply(lambda subdf: subdf.iloc[-1]).reset_index(drop=True).set_index(['pat_id','adm_index']).sort_index(),
              'first admission':lambda df: df.loc[idx[:,1],:],
              'second admission':lambda df: df.loc[idx[:,2],:],
              'third admission':lambda df: df.loc[idx[:,3],:],
              '>4 admission':lambda df: df.loc[idx[:,4:],:],
              'all':lambda df: df
             }
data_sets = {'train': {'outs':train_outs,'golden':train_golden},
             'val': {'outs':val_outs,'golden':val_golden},
             'test':{'outs':test_outs,'golden':test_golden}
            }

metrics = {data_set:
           {
               name: compute_metrics(model_outputs=operations[name](data_sets[data_set]['outs']),
                                     model_predictions=None,
                                     golden=operations[name](data_sets[data_set]['golden']),
                                     metrics=['recall@30','precision@30']
                                    ).filter(like='_adm')
               for name in tqdm(operations)
           }
           for data_set in tqdm(data_sets)
          }

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

In [183]:
res = pd.concat([ pd.concat([pd.concat([metrics[data_set][level].rename(level) for level in operations],axis=1)],axis=1,keys=[data_set]) for data_set in data_sets],axis=1).T

In [184]:
res

metrics                 recall@30_adm  precision@30_adm
train last admission      0.745267         0.289420    
      first admission     0.741529         0.271449    
      second admission    0.762765         0.313066    
      third admission     0.768344         0.341089    
      >4 admission        0.796499         0.361650    
      all                 0.754284         0.295720    
val   last admission      0.716719         0.269333    
      first admission     0.718916         0.247941    
      second admission    0.716869         0.289478    
      third admission     0.726795         0.319099    
      >4 admission        0.742629         0.339944    
      all                 0.722141         0.274404    
test  last admission      0.702728         0.274726    
      first admission     0.702754         0.259289    
      second admission    0.702626         0.297207    
      third admission     0.720550         0.327137    
      >4 admission        0.735542         0.325244    
      all                 0.708381         0.280547

In [169]:
a = pd.DataFrame([[1,2],[3,4]],columns=['a','b'])
b = pd.DataFrame([[5,6],[7,8]],columns=['a','b'])
pd.concat([a,b])

,a,b
0,1,2
1,3,4
0,5,6
1,7,8


In [156]:
metrics['train'].keys()

dict_keys(['last admission', 'first admission', 'second admission', 'third admission', '>4 admission', 'all'])

In [148]:
metrics.keys()

dict_keys([<rnn_utils.DiagnosesDataset object at 0x7fcf61ebb3d0>])

In [145]:
from tqdm.notebook import tqdm

In [ ]:
metrics.keys()

In [141]:
train_golden_levels['last admission'].shape

(5249, 272)

In [137]:
train_outs_levels['last admission'].

Int64Index([   17,    34,    36,    67,    68,    84,    85,   103,   107,
              112,
            ...
            99562, 99613, 99660, 99712, 99781, 99783, 99822, 99883, 99897,
            99982],
           dtype='int64', name='pat_id', length=5249)

In [136]:
train_golden.loc[train_outs_levels['last admission'].index]

diag_0  diag_1  diag_2  diag_3  diag_4  diag_5  diag_6  \
pat_id adm_index                                                           
17     1             0.0     0.0     0.0     0.0     0.0     0.0     0.0   
34     1             0.0     0.0     0.0     0.0     0.0     0.0     0.0   
36     1             0.0     0.0     0.0     0.0     0.0     0.0     0.0   
       2             0.0     0.0     0.0     0.0     0.0     0.0     0.0   
67     1             0.0     0.0     0.0     0.0     0.0     0.0     0.0   
...                  ...     ...     ...     ...     ...     ...     ...   
99822  2             0.0     1.0     0.0     0.0     0.0     0.0     0.0   
99883  1             0.0     0.0     0.0     0.0     0.0     0.0     0.0   
99897  1             0.0     0.0     0.0     0.0     0.0     0.0     0.0   
99982  1             0.0     0.0     0.0     0.0     0.0     0.0     0.0   
       2             0.0     0.0     0.0     0.0     0.0     0.0     0.0   

                  diag_7  diag_8  diag_9  ...  diag_262  diag_263  diag_264  \
pat_id adm_index                          ...                                 
17     1             0.0     0.0     0.0  ...       0.0       0.0       0.0   
34     1             0.0     0.0     0.0  ...       0.0       0.0       0.0   
36     1             0.0     0.0     0.0  ...       0.0       0.0       0.0   
       2             0.0     0.0     0.0  ...       0.0       0.0       0.0   
67     1             0.0     0.0     0.0  ...       0.0       0.0       0.0   
...                  ...     ...     ...  ...       ...       ...       ...   
99822  2             0.0     0.0     0.0  ...       0.0       0.0       0.0   
99883  1             0.0     0.0     0.0  ...       0.0       0.0       0.0   
99897  1             0.0     0.0     1.0  ...       0.0       0.0       0.0   
99982  1             0.0     0.0     0.0  ...       0.0       0.0       0.0   
       2             0.0     0.0     0.0  ...       0.0       0.0       0.0   

                  diag_265  diag_266  diag_267  diag_268  diag_269  diag_270  \
pat_id adm_index                                                               
17     1               0.0       0.0       0.0       0.0       0.0       0.0   
34     1               0.0       0.0       0.0       0.0       0.0       0.0   
36     1               0.0       0.0       0.0       0.0       0.0       0.0   
       2               0.0       0.0       0.0       0.0       0.0       0.0   
67     1               0.0       0.0       0.0       0.0       0.0       0.0   
...                    ...       ...       ...       ...       ...       ...   
99822  2               0.0       0.0       1.0       0.0       0.0       0.0   
99883  1               0.0       0.0       0.0       0.0       0.0       0.0   
99897  1               0.0       0.0       0.0       0.0       0.0       0.0   
99982  1               0.0       0.0       0.0       0.0       0.0       0.0   
       2               0.0       0.0       0.0       0.0       0.0       0.0   

                  diag_271  
pat_id adm_index            
17     1               0.0  
34     1               0.0  
36     1               0.0  
       2               0.0  
67     1               0.0  
...                    ...  
99822  2               0.0  
99883  1               0.0  
99897  1               0.0  
99982  1               0.0  
       2               0.0  

[8591 rows x 272 columns]

In [112]:
metrics_levels ={name:compute_metrics(outs_levels[name],None, for name in outs_levels.keys()

,,diag_0,diag_1,diag_2,diag_3,diag_4,diag_5,diag_6,diag_7,diag_8,diag_9,...,diag_262,diag_263,diag_264,diag_265,diag_266,diag_267,diag_268,diag_269,diag_270,diag_271
pat_id,adm_index,,,,,,,,,,,,,,,,,,,,,
36,2,0.000673,0.309826,0.137909,0.039800,0.000030,0.000999,0.003343,0.000968,0.000547,0.018139,...,0.000475,0.001987,0.000211,0.009656,0.177691,0.055659,0.001503,0.000787,0.007322,0.036337
107,2,0.004353,0.281784,0.185377,0.072717,0.003263,0.015787,0.009231,0.019405,0.001721,0.025043,...,0.000352,0.001791,0.000861,0.003919,0.376343,0.101457,0.007359,0.001163,0.008888,0.090163
138,2,0.001536,0.175478,0.094343,0.038436,0.001288,0.010362,0.010066,0.004306,0.001001,0.009868,...,0.000218,0.000981,0.000367,0.002564,0.166460,0.098313,0.000761,0.000959,0.004777,0.035342
154,2,0.000267,0.098016,0.072805,0.023871,0.000033,0.001566,0.003745,0.000918,0.000612,0.006274,...,0.000079,0.000920,0.000254,0.001302,0.173248,0.058742,0.001326,0.000805,0.005788,0.030788
188,2,0.000883,0.276198,0.118234,0.071679,0.002264,0.296970,0.012145,0.045971,0.000349,0.029990,...,0.001332,0.003944,0.001025,0.001759,0.206453,0.046278,0.000564,0.004154,0.007653,0.065389
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99346,2,0.000988,0.291150,0.148026,0.069000,0.000073,0.002416,0.013226,0.006024,0.000494,0.021715,...,0.000318,0.003025,0.000391,0.004811,0.098054,0.158576,0.001952,0.000648,0.010453,0.061084
99439,2,0.005030,0.217749,0.231138,0.041940,0.000089,0.000563,0.002751,0.003461,0.000220,0.027295,...,0.000084,0.002198,0.000114,0.013327,0.073613,0.103881,0.000777,0.000337,0.005469,0.017529
99503,2,0.001304,0.234582,0.129450,0.041763,0.001145,0.005442,0.005378,0.012484,0.001163,0.050323,...,0.000146,0.001179,0.000249,0.001741,0.394561,0.129328,0.001115,0.000507,0.007033,0.058014


In [111]:
last_adm

,diag_0,diag_1,diag_2,diag_3,diag_4,diag_5,diag_6,diag_7,diag_8,diag_9,...,diag_262,diag_263,diag_264,diag_265,diag_266,diag_267,diag_268,diag_269,diag_270,diag_271
pat_id,,,,,,,,,,,,,,,,,,,,,
17,0.001214,0.108292,0.076243,0.029526,0.000564,0.013792,0.013004,0.010701,0.001110,0.025458,...,0.000399,0.002593,0.001516,0.002587,0.183489,0.100942,0.003286,0.002262,0.016034,0.030779
34,0.001015,0.139923,0.056569,0.025739,0.000340,0.003556,0.005176,0.002287,0.000618,0.005535,...,0.000193,0.001326,0.000955,0.002076,0.128642,0.077942,0.002250,0.000594,0.005766,0.026409
36,0.000673,0.309826,0.137909,0.039800,0.000030,0.000999,0.003343,0.000968,0.000547,0.018139,...,0.000475,0.001987,0.000211,0.009656,0.177691,0.055659,0.001503,0.000787,0.007322,0.036337
67,0.000562,0.191483,0.115520,0.034675,0.000317,0.023222,0.015298,0.005761,0.000895,0.009938,...,0.000633,0.008380,0.001832,0.003482,0.119236,0.063102,0.009220,0.002443,0.016113,0.027118
68,0.018426,0.336105,0.090279,0.116326,0.892153,0.174349,0.055179,0.084393,0.001569,0.019454,...,0.000453,0.006399,0.000915,0.002709,0.056248,0.062515,0.002630,0.000896,0.004166,0.012311
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99783,0.000619,0.132416,0.111008,0.019738,0.000039,0.003323,0.008135,0.005515,0.000483,0.016922,...,0.000280,0.001178,0.000660,0.003850,0.209524,0.097796,0.001987,0.001381,0.011780,0.029831
99822,0.018709,0.302541,0.224977,0.241892,0.001645,0.203377,0.046442,0.081892,0.000194,0.015038,...,0.002293,0.010966,0.000975,0.031630,0.177031,0.179438,0.014060,0.003243,0.021491,0.087465
99883,0.005573,0.166262,0.104042,0.062963,0.002047,0.006085,0.012752,0.010384,0.000426,0.008506,...,0.000251,0.001905,0.000457,0.010676,0.078702,0.077005,0.005149,0.000355,0.003849,0.015750


In [105]:
mask = train_outs.groupby('pat_id').apply(lambda x: x.adm_index == 1).droplevel(level=1)
mask.head(5)

pat_id
17     True
34     True
36     True
36    False
67     True
Name: adm_index, dtype: bool

In [101]:
mask.index

MultiIndex([(   17,    0),
            (   34,    1),
            (   36,    2),
            (   36,    3),
            (   67,    4),
            (   68,    5),
            (   84,    6),
            (   85,    7),
            (  103,    8),
            (  107,    9),
            ...
            (99660, 8581),
            (99712, 8582),
            (99781, 8583),
            (99783, 8584),
            (99822, 8585),
            (99822, 8586),
            (99883, 8587),
            (99897, 8588),
            (99982, 8589),
            (99982, 8590)],
           names=['pat_id', None], length=8591)

In [93]:
train_outs.loc[mask]

/tmp/ipykernel_6013/1371323828.py:1: FutureWarning: reindexing with a non-unique Index is deprecated and will raise in a future version.
  train_outs.loc[mask]


ValueError: cannot reindex on an axis with duplicate labels

In [ ]:
train_outs
                    .reset_index()
                    .groupby('pat_id')
                    .adm_index == 1)
                    .to_frame()
                    .reset_index()
                    .set_index(['pat_id','adm_index'])
                    .index

In [72]:
idx = pd.IndexSlice
last_adm_index = (train_outs
                 .reset_index()
                 .groupby('pat_id')
                 .adm_index.max()
                 .to_frame()
                 .reset_index()
                 .set_index(['pat_id','adm_index'])
                 .index
                )

first_adm_index = ((train_outs
                    .reset_index()
                    .groupby('pat_id')
                    .adm_index == 1)
                    .to_frame()
                    .reset_index()
                    .set_index(['pat_id','adm_index'])
                    .index
                  )
second_adm_index = ((train_outs
                    .reset_index()
                    .groupby('pat_id')
                    .adm_index == 2)
                    .to_frame()
                    .reset_index()
                    .set_index(['pat_id','adm_index'])
                    .index
                  )
third_adm_index = ((train_outs
                    .reset_index()
                    .groupby('pat_id')
                    .adm_index == 3)
                    .to_frame()
                    .reset_index()
                    .set_index(['pat_id','adm_index'])
                    .index
                  )
over4_adm_index = ((train_outs
                    .reset_index()
                    .groupby('pat_id')
                    .adm_index >= 4)
                    .to_frame()
                    .reset_index()
                    .set_index(['pat_id','adm_index'])
                    .index
                  )
#train_outs.iloc[:1000].groupby(['pat_id']).apply(lambda subdf:subdf.iloc[-1])

AttributeError: 'bool' object has no attribute 'to_frame'

In [65]:
pd.MultiIndex.from_arrays(outs_last_adm.to_frame().reset_index().values)

MultiIndex([(17, 34, 36, 67, 68, 84, 85, 103, 107, 112, 117, 130, 135, ...),
            ( 1,  1,  2,  1,  1,  1,  1,   1,   2,   1,   1,   1,   1, ...)],
           )

In [70]:
outs_last_adm.to_frame().reset_index().set_index(['pat_id','adm_index']).index

MultiIndex([(   17, 1),
            (   34, 1),
            (   36, 2),
            (   67, 1),
            (   68, 1),
            (   84, 1),
            (   85, 1),
            (  103, 1),
            (  107, 2),
            (  112, 1),
            ...
            (99562, 1),
            (99613, 1),
            (99660, 1),
            (99712, 1),
            (99781, 1),
            (99783, 1),
            (99822, 2),
            (99883, 1),
            (99897, 1),
            (99982, 2)],
           names=['pat_id', 'adm_index'], length=5249)

In [71]:
train_outs.loc[idx[outs_last_adm.to_frame().reset_index().set_index(['pat_id','adm_index']).index],:]

,,diag_0,diag_1,diag_2,diag_3,diag_4,diag_5,diag_6,diag_7,diag_8,diag_9,...,diag_262,diag_263,diag_264,diag_265,diag_266,diag_267,diag_268,diag_269,diag_270,diag_271
pat_id,adm_index,,,,,,,,,,,,,,,,,,,,,
17,1,0.001214,0.108292,0.076243,0.029526,0.000564,0.013792,0.013004,0.010701,0.001110,0.025458,...,0.000399,0.002593,0.001516,0.002587,0.183489,0.100942,0.003286,0.002262,0.016034,0.030779
34,1,0.001015,0.139923,0.056569,0.025739,0.000340,0.003556,0.005176,0.002287,0.000618,0.005535,...,0.000193,0.001326,0.000955,0.002076,0.128642,0.077942,0.002250,0.000594,0.005766,0.026409
36,2,0.000673,0.309826,0.137909,0.039800,0.000030,0.000999,0.003343,0.000968,0.000547,0.018139,...,0.000475,0.001987,0.000211,0.009656,0.177691,0.055659,0.001503,0.000787,0.007322,0.036337
67,1,0.000562,0.191483,0.115520,0.034675,0.000317,0.023222,0.015298,0.005761,0.000895,0.009938,...,0.000633,0.008380,0.001832,0.003482,0.119236,0.063102,0.009220,0.002443,0.016113,0.027118
68,1,0.018426,0.336105,0.090279,0.116326,0.892153,0.174349,0.055179,0.084393,0.001569,0.019454,...,0.000453,0.006399,0.000915,0.002709,0.056248,0.062515,0.002630,0.000896,0.004166,0.012311
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99783,1,0.000619,0.132416,0.111008,0.019738,0.000039,0.003323,0.008135,0.005515,0.000483,0.016922,...,0.000280,0.001178,0.000660,0.003850,0.209524,0.097796,0.001987,0.001381,0.011780,0.029831
99822,2,0.018709,0.302541,0.224977,0.241892,0.001645,0.203377,0.046442,0.081892,0.000194,0.015038,...,0.002293,0.010966,0.000975,0.031630,0.177031,0.179438,0.014060,0.003243,0.021491,0.087465
99883,1,0.005573,0.166262,0.104042,0.062963,0.002047,0.006085,0.012752,0.010384,0.000426,0.008506,...,0.000251,0.001905,0.000457,0.010676,0.078702,0.077005,0.005149,0.000355,0.003849,0.015750


In [64]:

outs_last_adm.to_frame().reset_index().values

array([[   17,     1],
       [   34,     1],
       [   36,     2],
       ...,
       [99883,     1],
       [99897,     1],
       [99982,     2]])

In [14]:
train_outs.loc[idx

diag_0    diag_1    diag_2    diag_3    diag_4    diag_5  \
pat_id adm_index                                                               
17     1          0.001214  0.108292  0.076243  0.029526  0.000564  0.013792   
34     1          0.001015  0.139923  0.056569  0.025739  0.000340  0.003556   
36     1          0.001069  0.225121  0.101295  0.036027  0.000125  0.002095   
       2          0.000673  0.309826  0.137909  0.039800  0.000030  0.000999   
67     1          0.000562  0.191483  0.115520  0.034675  0.000317  0.023222   
...                    ...       ...       ...       ...       ...       ...   
99822  2          0.018709  0.302541  0.224977  0.241892  0.001645  0.203377   
99883  1          0.005573  0.166262  0.104042  0.062963  0.002047  0.006085   
99897  1          0.001253  0.114784  0.078070  0.036575  0.003091  0.005066   
99982  1          0.003874  0.156268  0.065483  0.035139  0.000614  0.004509   
       2          0.004502  0.116945  0.052524  0.028218  0.000466  0.002033   

                    diag_6    diag_7    diag_8    diag_9  ...  diag_262  \
pat_id adm_index                                          ...             
17     1          0.013004  0.010701  0.001110  0.025458  ...  0.000399   
34     1          0.005176  0.002287  0.000618  0.005535  ...  0.000193   
36     1          0.003853  0.001133  0.000424  0.011445  ...  0.000394   
       2          0.003343  0.000968  0.000547  0.018139  ...  0.000475   
67     1          0.015298  0.005761  0.000895  0.009938  ...  0.000633   
...                    ...       ...       ...       ...  ...       ...   
99822  2          0.046442  0.081892  0.000194  0.015038  ...  0.002293   
99883  1          0.012752  0.010384  0.000426  0.008506  ...  0.000251   
99897  1          0.013815  0.014786  0.003441  0.019384  ...  0.000124   
99982  1          0.008901  0.006822  0.000385  0.005916  ...  0.000263   
       2          0.005742  0.015931  0.000181  0.013844  ...  0.000223   

                  diag_263  diag_264  diag_265  diag_266  diag_267  diag_268  \
pat_id adm_index                                                               
17     1          0.002593  0.001516  0.002587  0.183489  0.100942  0.003286   
34     1          0.001326  0.000955  0.002076  0.128642  0.077942  0.002250   
36     1          0.002176  0.000428  0.004949  0.149484  0.042967  0.001551   
       2          0.001987  0.000211  0.009656  0.177691  0.055659  0.001503   
67     1          0.008380  0.001832  0.003482  0.119236  0.063102  0.009220   
...                    ...       ...       ...       ...       ...       ...   
99822  2          0.010966  0.000975  0.031630  0.177031  0.179438  0.014060   
99883  1          0.001905  0.000457  0.010676  0.078702  0.077005  0.005149   
99897  1          0.005626  0.000620  0.001870  0.068608  0.073924  0.001121   
99982  1          0.001019  0.000553  0.004827  0.109606  0.109890  0.001037   
       2          0.000682  0.000265  0.003914  0.062224  0.086466  0.000299   

                  diag_269  diag_270  diag_271  
pat_id adm_index                                
17     1          0.002262  0.016034  0.030779  
34     1          0.000594  0.005766  0.026409  
36     1          0.000820  0.005606  0.025454  
       2          0.000787  0.007322  0.036337  
67     1          0.002443  0.016113  0.027118  
...                    ...       ...       ...  
99822  2          0.003243  0.021491  0.087465  
99883  1          0.000355  0.003849  0.015750  
99897  1          0.000975  0.004625  0.019549  
99982  1          0.000442  0.006300  0.017760  
       2          0.000232  0.005963  0.013664  

[8591 rows x 272 columns]

In [ ]:
mask_train_last_adm = train_outs

In [17]:
compute_metrics(outs,None,golden,['recall@30','precision@30'])

AttributeError: 'NoneType' object has no attribute 'columns'

In [ ]:
loss,metrics = eval_model(model,val_dataloader,dataset, thresholds,metrics=['recall@30','precision@30'],name='train_all')

In [15]:
metrics

metrics
recall_diag       0.511542
recall@30_diag    0.288743
recall_adm        0.656954
recall@30_adm     0.722141
dtype: float64

In [22]:
train_all = eval_model(model,train_dataloader,dataset, decision_thresholds=None,metrics=['recall@30','precision@30'],name='train_all')

0it [00:00, ?it/s]

1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4
1
2
3
4


AttributeError: 'NoneType' object has no attribute 'loc'

In [12]:
train_all = eval_model(model,train_dataloader,dataset, decision_thresholds=None,metrics=['recall@30','precision@30'], epoch, 'train')
val_results = eval_model(model,val_dataloader,dataset, criterion, epoch, 'validation')
test_results = eval_model(model,test_dataloader,dataset, criterion, epoch, 'test')
res = {'train_loss':train_results['loss'],
       'train_recall@30':train_results['last adm']['recall30']['mean'],
       'val_loss':val_results['loss'],
       'test_loss':test_results['loss'],
       'recall@30_test':test_results['last adm']['recall30']['mean'],
       'recall@10':val_results['last adm']['recall10']['mean'],
       'recall@20':val_results['last adm']['recall20']['mean'],
       'recall@30':val_results['last adm']['recall30']['mean'],
       'n_val_recall@30_last':val_results['last adm']['recall30']['n']
      }
res

/Users/simaonovais/opt/miniconda3/envs/thesis/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/simaonovais/opt/miniconda3/envs/thesis/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/simaonovais/opt/miniconda3/envs/thesis/lib/python3.9/site-packages/numpy/core/_methods.py:262: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/simaonovais/opt/miniconda3/envs/thesis/lib/python3.9/site-packages/numpy/core/_methods.py:222: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
/Users/simaonovais/opt/miniconda3/envs/thesis/lib/python3.9/site-packages/numpy/core/_methods.py:254: RuntimeWarning: invalid value encountered in double_scalars
  

{'train_loss': 0.09286758683197474,
 'train_recall@30': 0.7970463961291864,
 'val_loss': 0.11220189078648884,
 'test_loss': 0.11535388418038686,
 'recall@30_test': 0.7122651825787929,
 'recall@10': 0.4521770107365368,
 'recall@20': 0.619266100780294,
 'recall@30': 0.7185501911451649,
 'n_val_recall@30_last': 1125}

## Save model weights and parameters

In [12]:
model_folder = os.path.join(settings.data_base,settings.models_folder,model_name)

if not os.path.isdir(model_folder):
    os.mkdir(model_folder)

# save weights
weights_save_path = os.path.join(model_folder,"weights")

torch.save(model.state_dict(), 
           weights_save_path
          )
print('Model saved!')

Model saved!


In [20]:
import re
s = 'recall@30'
re.findall('(\w+)@\d+',s)

['recall']

## Save model hyperparameters

In [13]:
params = dict(input_size = input_size,
              hidden_size=hidden_size,
              num_layers=num_layers,
              n_labels=n_labels,
              model=rnn_type
             )


hypp_save_path = os.path.join(model_folder, 'hyper_parameters.json')

with open(hypp_save_path, "w") as f:
    json.dump(params, f)
    
print('Hyperparameters saved!')

Hyperparameters saved!


## Other parameters

In [14]:
other_params = dict(batch_size=batch_size,
                    grouping=grouping,
                    epochs=epochs
                   )

other_params_save_path = os.path.join(model_folder, 'other_params.json')

with open(other_params_save_path, "w") as f:
    json.dump(other_params, f)
    
print('Other parameters saved!')

Other parameters saved!


# Test it out

### Read weights and hyperparameters

In [15]:
#hyperparameters
with open(hypp_save_path,'r') as f:
    params_loaded = json.load(f)
    
# weights
weights = torch.load(weights_save_path)

### Create model and load weights

In [16]:
new_model = RNN(**params_loaded)
new_model.load_state_dict(torch.load(weights_save_path))

<All keys matched successfully>

### Evaluate it

In [18]:
eval_model(new_model,val_dataloader,dataset, criterion, epoch, 'validation')['last adm']

{'recall10': {'mean': 0.4521770107365368,
  'std': 0.20384737052487178,
  'n': 1125},
 'recall20': {'mean': 0.619266100780294,
  'std': 0.19393583573690798,
  'n': 1125},
 'recall30': {'mean': 0.7185501911451649, 'std': 0.17084153998634, 'n': 1125}}

# Performance Matches!